<a href="https://colab.research.google.com/github/ipeirotis-org/datasets/blob/main/Collisions/collisions_refactored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NYC Motor Vehicle Collisions Data Pipeline

This notebook downloads, cleans, geocodes, and uploads NYC collision data to Google BigQuery.

## Data Source
[NYC Open Data - Motor Vehicle Collisions - Crashes](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95)

## Output
Two BigQuery tables in `nyu-datasets.collisions`:
- **`collisions`**: Main collision records with location, time, and casualty information (~1.9M rows)
- **`causes_types`**: Vehicle contributing factors and types, linked by UNIQUE_KEY (~3.6M rows)

## Pipeline Steps
1. Download raw data from NYC Open Data API
2. Clean and standardize column names
3. Convert data types (dates, numerics)
4. Normalize vehicle types and contributing factors
5. Geocode locations using NYC shapefiles (ZIP codes, boroughs, neighborhoods)
6. Detect and filter data quality issues
7. Upload to BigQuery with proper schema definitions


In [ ]:
# =============================================================================
# Setup and Authentication
# =============================================================================
# Install required Google Cloud packages
!pip install -q google-cloud-secret-manager google-cloud-bigquery

# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

# Start timing the pipeline
from datetime import datetime
t_start = datetime.now()
print(f"Pipeline started at: {t_start}")

In [ ]:
# =============================================================================
# Configuration
# =============================================================================
# Render plots inline with retina display
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# BigQuery settings
PROJECT_ID = "nyu-datasets"
DATASET_ID = "collisions"

# Data source
DATA_URL = "https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD"


## 1. Download Raw Data

In [ ]:
# Download collision data from NYC Open Data
!curl 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD' -o accidents.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data - use object dtype initially to avoid type inference issues
df = pd.read_csv("accidents.csv", low_memory=False, dtype='object')

# Store original count for later statistics
ORIGINAL_RECORD_COUNT = len(df)

print(f"Loaded {ORIGINAL_RECORD_COUNT:,} collision records")
print(f"Columns: {len(df.columns)}")
df.head()

### Initial Data Exploration

Before cleaning, let's examine the raw data structure and identify potential issues.

In [ ]:
# Examine data types - all loaded as object initially
print("Column data types:")
print(df.dtypes)
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

In [ ]:
# Check for missing values in key columns
key_columns = ['COLLISION_ID', 'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE']
print("Missing values in key columns:")
for col in key_columns:
    if col in df.columns:
        missing = df[col].isna().sum()
        pct = 100 * missing / len(df)
        print(f"  {col}: {missing:,} ({pct:.1f}%)")

In [ ]:
# Borough distribution
print("Borough distribution:")
df['BOROUGH'].value_counts(dropna=False)

In [ ]:
# Date range check
print("Date range in raw data:")
print(f"  First date: {df['CRASH DATE'].min()}")
print(f"  Last date: {df['CRASH DATE'].max()}")

## 2. Data Cleaning and Type Conversion

In [ ]:
# Rename columns for consistency and clarity
column_mapping = {
    "COLLISION_ID": "UNIQUE_KEY",
    "ZIP CODE": "ZIPCODE",
    "NUMBER OF PERSONS INJURED": "PERSONS_INJURED",
    "NUMBER OF PERSONS KILLED": "PERSONS_KILLED",
    "NUMBER OF PEDESTRIANS INJURED": "PEDESTRIANS_INJURED",
    "NUMBER OF PEDESTRIANS KILLED": "PEDESTRIANS_KILLED",
    "NUMBER OF MOTORIST INJURED": "MOTORISTS_INJURED",
    "NUMBER OF MOTORIST KILLED": "MOTORISTS_KILLED",
    "NUMBER OF CYCLIST INJURED": "CYCLISTS_INJURED",
    "NUMBER OF CYCLIST KILLED": "CYCLISTS_KILLED",
    "CONTRIBUTING FACTOR VEHICLE 1": "CAUSE_VEHICLE_1",
    "CONTRIBUTING FACTOR VEHICLE 2": "CAUSE_VEHICLE_2",
    "CONTRIBUTING FACTOR VEHICLE 3": "CAUSE_VEHICLE_3",
    "CONTRIBUTING FACTOR VEHICLE 4": "CAUSE_VEHICLE_4",
    "CONTRIBUTING FACTOR VEHICLE 5": "CAUSE_VEHICLE_5",
    "VEHICLE TYPE CODE 1": "TYPE_VEHICLE_1",
    "VEHICLE TYPE CODE 2": "TYPE_VEHICLE_2",
    "VEHICLE TYPE CODE 3": "TYPE_VEHICLE_3",
    "VEHICLE TYPE CODE 4": "TYPE_VEHICLE_4",
    "VEHICLE TYPE CODE 5": "TYPE_VEHICLE_5",
}
df = df.rename(columns=column_mapping)

# Replace spaces with underscores in remaining column names
df.columns = df.columns.str.replace(' ', '_')
print("Columns after renaming:")
print(df.columns.tolist())

### DateTime Conversion

In [ ]:
# Combine date and time into a single datetime column
df['DATE_TIME'] = pd.to_datetime(
    df['CRASH_DATE'] + ' ' + df['CRASH_TIME'],
    format="%m/%d/%Y %H:%M"
)
print(f"Date range: {df['DATE_TIME'].min()} to {df['DATE_TIME'].max()}")

### Numeric Fields

In [ ]:
# Convert casualty columns to numeric (consolidated from 8 separate cells)
numeric_columns = [
    'PERSONS_INJURED', 'PERSONS_KILLED',
    'PEDESTRIANS_INJURED', 'PEDESTRIANS_KILLED',
    'CYCLISTS_INJURED', 'CYCLISTS_KILLED',
    'MOTORISTS_INJURED', 'MOTORISTS_KILLED'
]

for col in numeric_columns:
    df[col] = pd.to_numeric(df[col].fillna(0), downcast='unsigned')

print("Numeric column stats:")
df[numeric_columns].describe()

### Latitude and Longitude

In [ ]:
# Handle missing coordinates
df['LATITUDE'] = pd.to_numeric(df['LATITUDE'], errors='coerce').fillna(0.0)
df['LONGITUDE'] = pd.to_numeric(df['LONGITUDE'], errors='coerce').fillna(0.0)

# Create LOCATION string for reference
df['LOCATION'] = '(' + df['LATITUDE'].astype(str) + ', ' + df['LONGITUDE'].astype(str) + ')'

print(f"Records with valid coordinates: {(df['LATITUDE'] != 0).sum():,}")
print(f"Records missing coordinates: {(df['LATITUDE'] == 0).sum():,}")

## 3. Normalize Vehicle Causes and Types

In [ ]:
# Normalize cause and type columns to uppercase
cause_type_cols = [
    'CAUSE_VEHICLE_1', 'CAUSE_VEHICLE_2', 'CAUSE_VEHICLE_3', 'CAUSE_VEHICLE_4', 'CAUSE_VEHICLE_5',
    'TYPE_VEHICLE_1', 'TYPE_VEHICLE_2', 'TYPE_VEHICLE_3', 'TYPE_VEHICLE_4', 'TYPE_VEHICLE_5'
]

for col in cause_type_cols:
    df[col] = df[col].str.upper()

### Create Normalized Causes/Types Table

Unpivot the 5 vehicle columns into a single table with one row per vehicle involved in each collision.

In [ ]:
# Create a normalized table for causes and vehicle types
# This unpivots the 5 vehicle columns into rows

vehicle_dfs = []
for i in range(1, 6):
    temp_df = df[['UNIQUE_KEY', f'CAUSE_VEHICLE_{i}', f'TYPE_VEHICLE_{i}']].copy()
    temp_df['VEHICLE'] = i
    temp_df.columns = ['UNIQUE_KEY', 'CAUSE', 'VEHICLE_TYPE', 'VEHICLE']
    vehicle_dfs.append(temp_df)

ct_df = pd.concat(vehicle_dfs, ignore_index=True)
ct_df = ct_df.dropna(subset=['CAUSE', 'VEHICLE_TYPE'], how='all')
ct_df = ct_df.sort_values(['UNIQUE_KEY', 'VEHICLE'])

print(f"Created causes_types table: {len(ct_df):,} rows")
ct_df.head(10)

### Standardize Vehicle Types

The raw data contains many variations and typos for vehicle types. We normalize these to canonical values using a mapping dictionary.

Reference: [NY DMV Vehicle Body Type Codes](https://data.ny.gov/api/assets/83055271-29A6-4ED4-9374-E159F30DB5AE)

In [ ]:
# Vehicle type normalization mapping
# Maps variant spellings/abbreviations to canonical types
VEHICLE_TYPE_MAPPING = {
    'AMBULANCE': ['AM', 'AMB', 'AMBU', 'AMBUL', 'ANBUL', 'AMABU', 'AMULA', 'ABULA', 'AMBULANCE`', 'ALMBULANCE', 'AMUBULANCE', 'AMULANCE'],
    'BICYCLE': ['BICYC', 'BIKE'],
    'BOX TRUCK': ['FB', 'BOX T', 'BOX', 'BOXTR'],
    'BUS': ['BU', 'BS', 'SCHOOL BUS'],
    'COMMERCIAL': ['COMME', 'COM', 'COMM', 'COM T', 'COM.', 'COMMM', 'COMER', 'COMIX', 'COMPA', 'COMB', 'COMMU', 'COMM.', 'C0MME'],
    'CONVERTIBLE': ['CONV', 'CONVE'],
    'DELIVERY TRUCK': ['DELV', 'DEL'],
    'DUMP TRUCK': ['DUMP', 'DUMPS', 'DUMPT', 'PUMP', 'DP'],
    'E-BIKE': ['E BIK', 'E-BIK', 'E/BIK', 'EBIKE'],
    'E-SCOOTER': ['GARBAGE TR'],
    'FIRE TRUCK': ['FDNY', 'FIRE', 'FIRET', 'FD NY', 'FD TR'],
    'FIRETRUCK': ['FIRE TRUCK', 'FDNY TRUCK'],
    'FORK LIFT': ['FORKL', 'FORK'],
    'GARBAGE OR REFUSE': ['GARBA'],
    'LARGE COM VEH(6 OR MORE TIRES)': ['MULTI-WHEELED VEHICLE'],
    'LIVERY VEHICLE': ['LIVER', 'LIMOU', 'LIMO', 'LIMO/', 'OML/'],
    'MOTORCYCLE': ['MOTOR', 'MINICYCLE', 'MOTORBIKE'],
    'PASSENGER VEHICLE': ['PASS', 'PAS', 'PASSE'],
    'PEDICAB': ['PEDIC'],
    'PICK-UP TRUCK': ['TKP', 'PK', 'PICK', 'PICK-', 'PICKU', 'PICKUP WITH MOUNTED CAMPER', 'FLAT BED', 'FLAT RACK', 'FLATB', 'FLAT', 'BACK', 'PICK UP TR'],
    'POWER SHOVEL': ['P/SH'],
    'R/V': ['MOTOR HOME', 'MOTORIZED HOME', 'RV', 'R/V C', 'RV/TR', 'H/WH'],
    'ROAD SWEEPER': ['RD/S'],
    'SANITATION': ['DSNY', 'DS', 'SANIT'],
    'SCOOTER': ['SC', 'SCOO', 'SCOOT', 'MOTER', 'MOPD', 'MOPAD', 'MOPET', 'MOPEN', 'MINIBIKE', 'MOPED'],
    'SEDAN': ['2 DR SEDAN', '4 DR SEDAN', 'SUDAN', 'SE', '4DS', '4DSD', '3-DOOR'],
    'SMALL COM VEH(4 TIRES)': ['SMALL', 'SMALL COM VEH(4 TIRES) '],
    'SNOW PLOW': ['SP'],
    'SPORT UTILITY / STATION WAGON': ['STATION WAGON/SPORT UTILITY VEHICLE', 'SUBN', 'SUBN/', 'WAGON'],
    'TANK TRUCK': ['TANK', 'TANKE', 'TANKER'],
    'TOW TRUCK / WRECKER': ['TOW T', 'TOW', 'TOW TRUCK', 'TOWTR', 'TOW-T', 'TOWIN', 'TOWER', 'G TOW'],
    'TRACTOR TRUCK': ['TRACTOR TRUCK DIESEL', 'TRACTOR TRUCK GASOLINE', 'TRAC.', 'TRAC', 'TRACTOR TR', 'TRACTOR'],
    'TRAILER': ['TRAIL', 'TR', 'TRL', 'TRLR'],
    'TRUCK': ['SEMI', 'SEMI-', 'MACK', 'TK', 'TRACT', 'TRK', 'TRACK'],
    'UNKNOWN': ['OTHER', 'UNK', 'UNK,', 'UNKN', 'UNKNO', 'UNKNOWN', 'UNKOW', 'UNNKO'],
    'USPS': ['US PO', 'USPOS', 'USPS2', 'USPST', 'U.S P', 'U.S.', 'USPS TRUCK'],
    'VAN': ['VAN CAMPER', 'VAN T', 'VAN F', 'VAN A', 'VAN W', 'VAN/B', 'VAB', 'VANG', 'VAN C', 'VAN/T', 'VANETTE', 'VAN`', 'VAV', 'VN', 'VAN (', 'ENCLOSED BODY - REMOVABLE ENCLOSURE', 'ENCLOSED BODY - NONREMOVABLE ENCLOSURE', 'ENCLO'],
}

In [ ]:
# Apply vehicle type normalization
for canonical, variants in VEHICLE_TYPE_MAPPING.items():
    ct_df['VEHICLE_TYPE'] = ct_df['VEHICLE_TYPE'].replace(variants, canonical)

print(f"Unique vehicle types after normalization: {ct_df['VEHICLE_TYPE'].nunique()}")
print("\nTop 15 vehicle types:")
ct_df['VEHICLE_TYPE'].value_counts().head(15)

In [ ]:
# Drop the individual vehicle columns from main dataframe (now in ct_df)
vehicle_cols_to_drop = [
    'CAUSE_VEHICLE_1', 'TYPE_VEHICLE_1',
    'CAUSE_VEHICLE_2', 'TYPE_VEHICLE_2',
    'CAUSE_VEHICLE_3', 'TYPE_VEHICLE_3',
    'CAUSE_VEHICLE_4', 'TYPE_VEHICLE_4',
    'CAUSE_VEHICLE_5', 'TYPE_VEHICLE_5'
]
df = df.drop(columns=vehicle_cols_to_drop)
print(f"Main dataframe columns: {len(df.columns)}")

## 4. Data Quality Checks

In [ ]:
# Identify records with data quality issues

# 1. Incorrect sum of injured people
check_injured = (
    df.PEDESTRIANS_INJURED + df.CYCLISTS_INJURED + df.MOTORISTS_INJURED != df.PERSONS_INJURED
)
incorrect_injured = set(df[check_injured].UNIQUE_KEY.values)
print(f"Records with incorrect injured totals: {len(incorrect_injured):,}")

# 2. Incorrect sum of killed people
check_killed = (
    df.PEDESTRIANS_KILLED + df.CYCLISTS_KILLED + df.MOTORISTS_KILLED != df.PERSONS_KILLED
)
incorrect_killed = set(df[check_killed].UNIQUE_KEY.values)
print(f"Records with incorrect killed totals: {len(incorrect_killed):,}")

# 3. No vehicle/cause entries
nocause = set(df.UNIQUE_KEY.values) - set(ct_df.UNIQUE_KEY.values)
print(f"Records with no vehicle information: {len(nocause):,}")

# 4. Inconsistent vehicle numbering
vehicle_counts = ct_df.groupby('UNIQUE_KEY').agg(
    count=('VEHICLE', 'count'),
    max_num=('VEHICLE', 'max')
)
incorrect_vehicles = set(vehicle_counts[vehicle_counts['count'] != vehicle_counts['max_num']].index)
print(f"Records with vehicle numbering gaps: {len(incorrect_vehicles):,}")

# Combine all issues
records_to_exclude = incorrect_injured | incorrect_killed | nocause | incorrect_vehicles
print(f"\nTotal unique records with issues: {len(records_to_exclude):,}")

### Data Quality Visualization

Visualize the temporal distribution of data quality issues to understand if problems are systematic or random.

In [ ]:
# Plot data quality issues over time
# Create a dataframe of problematic records
problem_df = df[df['UNIQUE_KEY'].isin(records_to_exclude)].copy()

fig, ax = plt.subplots(figsize=(14, 4))
problem_df.set_index('DATE_TIME').resample('1W')['UNIQUE_KEY'].count().plot(
    ax=ax,
    title='Data Quality Issues by Week',
    ylabel='Number of problematic records'
)
plt.tight_layout()
plt.show()

print(f"\nTime period with most issues: {problem_df.set_index('DATE_TIME').resample('1M')['UNIQUE_KEY'].count().idxmax()}")

In [ ]:
# Breakdown of issue types
print("Issue Type Breakdown:")
print(f"  Incorrect injured totals: {len(incorrect_injured):,}")
print(f"  Incorrect killed totals:  {len(incorrect_killed):,}")
print(f"  Missing vehicle info:     {len(nocause):,}")
print(f"  Vehicle numbering gaps:   {len(incorrect_vehicles):,}")
print(f"  ─────────────────────────────────")
print(f"  Total unique (with overlap): {len(records_to_exclude):,}")

# Check overlap between categories
from itertools import combinations
categories = [
    ('incorrect_injured', incorrect_injured),
    ('incorrect_killed', incorrect_killed),
    ('nocause', nocause),
    ('incorrect_vehicles', incorrect_vehicles)
]
print("\nOverlap between categories:")
for (n1, s1), (n2, s2) in combinations(categories, 2):
    overlap = len(s1 & s2)
    if overlap > 0:
        print(f"  {n1} ∩ {n2}: {overlap:,}")

## 5. Geocoding with NYC Shapefiles

Use spatial joins with NYC shapefiles to:
1. Identify the neighborhood and borough for each collision location
2. Detect and fill missing ZIP codes
3. Filter out collisions outside NYC boundaries

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Convert collision coordinates to GeoDataFrame
geometry = [Point(xy) for xy in zip(df['LONGITUDE'].astype(float), df['LATITUDE'].astype(float))]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
print(f"Created GeoDataFrame with {len(gdf):,} records")

In [ ]:
%%time
# Load NYC neighborhood shapefile
shapefile_url = 'https://data.cityofnewyork.us/resource/9nt8-h7nd.geojson'
df_nyc = gpd.GeoDataFrame.from_file(shapefile_url)
df_nyc = df_nyc.to_crs("EPSG:4326")

print(f"Loaded {len(df_nyc)} NYC neighborhoods")
df_nyc[['ntaname', 'boro_name']].head()

In [ ]:
%%time
# Spatial join: match each collision with its neighborhood
# Using left join to preserve all collision records
gdf = gpd.sjoin(gdf, df_nyc[['ntaname', 'boro_name', 'geometry']], how='left', predicate='within')

# Rename columns for clarity
gdf = gdf.rename(columns={'ntaname': 'NEIGHBORHOOD', 'boro_name': 'DETECTED_BOROUGH'})

# Drop unnecessary columns from the join
if 'index_right' in gdf.columns:
    gdf = gdf.drop(columns=['index_right'])

print(f"Records matched to neighborhoods: {gdf['NEIGHBORHOOD'].notna().sum():,}")
print(f"Records outside NYC boundaries: {gdf['NEIGHBORHOOD'].isna().sum():,}")

In [ ]:
%%time
# Load NYC ZIP code shapefile
shapefile_zip = 'https://data.cityofnewyork.us/resource/pri4-ifjk.geojson'
df_zip = gpd.GeoDataFrame.from_file(shapefile_zip)
df_zip = df_zip.to_crs("EPSG:4326")

# Spatial join to get ZIP codes
gdf = gpd.sjoin(gdf, df_zip[['modzcta', 'geometry']], how='left', predicate='within')
gdf = gdf.rename(columns={'modzcta': 'DETECTED_ZIPCODE'})

if 'index_right' in gdf.columns:
    gdf = gdf.drop(columns=['index_right'])

print(f"Records matched to ZIP codes: {gdf['DETECTED_ZIPCODE'].notna().sum():,}")

### Post-Geocoding Analysis

Compare detected locations with originally reported values and identify discrepancies.

In [ ]:
# Analyze location detection results
print("=== Location Detection Summary ===")
print(f"Total records: {len(gdf):,}")
print(f"\nNeighborhood detection:")
print(f"  - Detected: {gdf['NEIGHBORHOOD'].notna().sum():,}")
print(f"  - Missing: {gdf['NEIGHBORHOOD'].isna().sum():,}")

print(f"\nBorough comparison:")
# Records where detected borough differs from reported
borough_mismatch = (
    gdf['DETECTED_BOROUGH'].notna() &
    gdf['BOROUGH'].notna() &
    (gdf['DETECTED_BOROUGH'].str.upper() != gdf['BOROUGH'].str.upper())
)
print(f"  - Mismatches: {borough_mismatch.sum():,}")

print(f"\nZIP code comparison:")
zip_mismatch = (
    gdf['DETECTED_ZIPCODE'].notna() &
    gdf['ZIPCODE'].notna() &
    (gdf['DETECTED_ZIPCODE'] != gdf['ZIPCODE'])
)
print(f"  - Mismatches: {zip_mismatch.sum():,}")

### Geocoding Deep Dive

Detailed analysis of geocoding results, identifying patterns in failures and discrepancies.

This section helps understand:
- Where geocoding fails (outside NYC, water, parks, etc.)
- Common locations with missing data
- Systematic discrepancies between reported and detected locations

In [ ]:
# Define NYC bounding box for filtering
# Approximate bounds: 40.4°N to 40.95°N, -74.3°W to -73.65°W
NYC_BOUNDS = {
    'lat_min': 40.4,
    'lat_max': 40.95,
    'lon_min': -74.3,
    'lon_max': -73.65
}

# Check which records fall outside NYC bounds
outside_nyc_mask = ~(
    (gdf['LATITUDE'] >= NYC_BOUNDS['lat_min']) &
    (gdf['LATITUDE'] <= NYC_BOUNDS['lat_max']) &
    (gdf['LONGITUDE'] >= NYC_BOUNDS['lon_min']) &
    (gdf['LONGITUDE'] <= NYC_BOUNDS['lon_max'])
)

# Records with no/zero coordinates
no_coords_mask = (gdf['LATITUDE'] == 0) | (gdf['LONGITUDE'] == 0)

print("=== Coordinate Analysis ===")
print(f"Records with zero coordinates: {no_coords_mask.sum():,}")
print(f"Records outside NYC bounds: {(outside_nyc_mask & ~no_coords_mask).sum():,}")
print(f"Records within NYC bounds: {(~outside_nyc_mask & ~no_coords_mask).sum():,}")

In [ ]:
# Analyze records outside NYC boundaries (excluding zero coords)
outside_records = gdf[outside_nyc_mask & ~no_coords_mask].copy()

if len(outside_records) > 0:
    print(f"\n=== Records Outside NYC Bounds ({len(outside_records):,}) ===")
    print("\nSample coordinates:")
    print(outside_records[['LATITUDE', 'LONGITUDE', 'BOROUGH', 'ON_STREET_NAME']].head(10))

    print("\nReported boroughs for outside-NYC records:")
    print(outside_records['BOROUGH'].value_counts(dropna=False).head())

In [ ]:
# Visualize collision density on NYC map
fig, ax = plt.subplots(figsize=(12, 12))

# Plot NYC neighborhoods as base map
df_nyc.plot(
    ax=ax,
    color='white',
    edgecolor='gray',
    linewidth=0.5,
    alpha=0.8
)

# Plot collisions that were successfully geocoded
valid_coords = gdf[
    (gdf['NEIGHBORHOOD'].notna()) &
    (gdf['LATITUDE'] != 0)
].copy()

valid_coords.plot(
    ax=ax,
    kind='scatter',
    x='LONGITUDE',
    y='LATITUDE',
    s=0.1,
    alpha=0.02,
    c='blue'
)

ax.set_title(f'NYC Motor Vehicle Collisions\n({len(valid_coords):,} geocoded records)', fontsize=14)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.tight_layout()
plt.show()

#### Records Without Detected Borough

These records have valid NYC coordinates but fall outside neighborhood polygons (water, parks, highway medians, etc.).

In [ ]:
# Records within NYC bounds but no borough detected
no_borough = gdf[
    ~outside_nyc_mask &
    ~no_coords_mask &
    gdf['NEIGHBORHOOD'].isna()
].copy()

print(f"=== No Borough Detected ({len(no_borough):,} records) ===")

if len(no_borough) > 0:
    print("\nMost common streets for unmatched locations:")
    print(no_borough['ON_STREET_NAME'].value_counts().head(15))

    print("\nMost common coordinate clusters:")
    # Round to 3 decimal places to find clusters
    no_borough['LOC_CLUSTER'] = (
        no_borough['LATITUDE'].round(3).astype(str) + ', ' +
        no_borough['LONGITUDE'].round(3).astype(str)
    )
    print(no_borough['LOC_CLUSTER'].value_counts().head(10))

In [ ]:
# Map of unmatched locations
if len(no_borough) > 0 and len(no_borough) < 50000:
    fig, ax = plt.subplots(figsize=(12, 12))

    df_nyc.plot(ax=ax, color='lightgray', edgecolor='gray', linewidth=0.5)

    no_borough.plot(
        ax=ax,
        kind='scatter',
        x='LONGITUDE',
        y='LATITUDE',
        s=1,
        alpha=0.3,
        c='red',
        label='No borough detected'
    )

    ax.set_title(f'Collisions Without Detected Borough ({len(no_borough):,} records)')
    ax.legend()
    plt.tight_layout()
    plt.show()

#### Borough Discrepancies

Cases where the detected borough differs from the reported borough. These could indicate:
- Data entry errors in the original report
- Boundary edge cases
- Geocoding precision issues

In [ ]:
# Analyze borough discrepancies
gdf['BOROUGH_UPPER'] = gdf['BOROUGH'].str.upper().str.strip()
gdf['DETECTED_UPPER'] = gdf['DETECTED_BOROUGH'].str.upper().str.strip()

borough_mismatch = gdf[
    gdf['BOROUGH_UPPER'].notna() &
    gdf['DETECTED_UPPER'].notna() &
    (gdf['BOROUGH_UPPER'] != gdf['DETECTED_UPPER'])
].copy()

print(f"=== Borough Discrepancies ({len(borough_mismatch):,} records) ===")

if len(borough_mismatch) > 0:
    # Cross-tabulation
    print("\nReported vs Detected Borough:")
    crosstab = pd.crosstab(
        borough_mismatch['BOROUGH_UPPER'],
        borough_mismatch['DETECTED_UPPER'],
        margins=True
    )
    print(crosstab)

    print("\nMost common mismatch locations:")
    borough_mismatch['LOC_CLUSTER'] = (
        borough_mismatch['LATITUDE'].round(3).astype(str) + ', ' +
        borough_mismatch['LONGITUDE'].round(3).astype(str)
    )
    print(borough_mismatch['LOC_CLUSTER'].value_counts().head(10))

In [ ]:
# Map of borough discrepancies
if len(borough_mismatch) > 0:
    fig, ax = plt.subplots(figsize=(12, 12))

    df_nyc.plot(ax=ax, color='white', edgecolor='black', linewidth=0.5)

    borough_mismatch.plot(
        ax=ax,
        kind='scatter',
        x='LONGITUDE',
        y='LATITUDE',
        s=2,
        alpha=0.5,
        c='orange',
        label='Borough mismatch'
    )

    ax.set_title(f'Borough Discrepancies ({len(borough_mismatch):,} records)\nReported ≠ Detected Borough')
    ax.legend()
    plt.tight_layout()
    plt.show()

#### ZIP Code Discrepancies

Similar analysis for ZIP code mismatches.

In [ ]:
# Analyze ZIP code discrepancies
zip_mismatch = gdf[
    gdf['ZIPCODE'].notna() &
    gdf['DETECTED_ZIPCODE'].notna() &
    (gdf['ZIPCODE'] != gdf['DETECTED_ZIPCODE'])
].copy()

print(f"=== ZIP Code Discrepancies ({len(zip_mismatch):,} records) ===")

if len(zip_mismatch) > 0:
    print("\nMost common ZIP mismatches (Reported → Detected):")
    zip_mismatch['ZIP_PAIR'] = zip_mismatch['ZIPCODE'] + ' → ' + zip_mismatch['DETECTED_ZIPCODE']
    print(zip_mismatch['ZIP_PAIR'].value_counts().head(15))

    print("\nMost common streets with ZIP discrepancies:")
    print(zip_mismatch['ON_STREET_NAME'].value_counts().head(10))

In [ ]:
# Summary statistics after geocoding analysis
print("=" * 60)
print("GEOCODING SUMMARY")
print("=" * 60)
print(f"\nTotal records: {len(gdf):,}")
print(f"\nCoordinate issues:")
print(f"  Zero coordinates: {no_coords_mask.sum():,}")
print(f"  Outside NYC bounds: {(outside_nyc_mask & ~no_coords_mask).sum():,}")
print(f"\nGeocoding results (for valid coordinates):")
print(f"  Successfully matched to neighborhood: {gdf['NEIGHBORHOOD'].notna().sum():,}")
print(f"  No neighborhood match: {(gdf['NEIGHBORHOOD'].isna() & ~no_coords_mask & ~outside_nyc_mask).sum():,}")
print(f"\nDiscrepancies:")
print(f"  Borough mismatches: {len(borough_mismatch):,}")
print(f"  ZIP code mismatches: {len(zip_mismatch):,}")

## 6. Prepare Final Dataset

Filter out problematic records and prepare the final columns for BigQuery upload.

In [ ]:
# Use detected values where available, fall back to reported values
gdf['ZIPCODE'] = gdf['DETECTED_ZIPCODE'].fillna(gdf['ZIPCODE'])
gdf['BOROUGH'] = gdf['DETECTED_BOROUGH'].fillna(gdf['BOROUGH'])

# Keep reported values for reference
gdf['REPORTED_ZIPCODE'] = df['ZIPCODE']
gdf['REPORTED_BOROUGH'] = df['BOROUGH']

# Filter to records within NYC boundaries (have a detected neighborhood)
gdf_nyc = gdf[gdf['NEIGHBORHOOD'].notna()].copy()
print(f"Records within NYC: {len(gdf_nyc):,}")

# Ensure ZIPCODE is 5 characters with leading zeros
gdf_nyc['ZIPCODE'] = gdf_nyc['ZIPCODE'].astype(str).str.zfill(5)

In [ ]:
# Remove records with data quality issues (identified earlier)
# Note: records_to_exclude was defined in the Data Quality section
final_df = gdf_nyc[~gdf_nyc['UNIQUE_KEY'].isin(records_to_exclude)].copy()

# Remove duplicate UNIQUE_KEYs (keep first occurrence)
duplicate_keys = final_df['UNIQUE_KEY'].value_counts()
duplicate_keys = duplicate_keys[duplicate_keys > 1].index
final_df = final_df[~final_df['UNIQUE_KEY'].isin(duplicate_keys)]

print(f"Final dataset: {len(final_df):,} collision records")

In [ ]:
# Select and order columns for the collisions table
collisions_columns = [
    'UNIQUE_KEY', 'DATE_TIME',
    'LATITUDE', 'LONGITUDE',
    'ZIPCODE', 'NEIGHBORHOOD', 'BOROUGH',
    'PERSONS_INJURED', 'PERSONS_KILLED',
    'PEDESTRIANS_INJURED', 'PEDESTRIANS_KILLED',
    'CYCLISTS_INJURED', 'CYCLISTS_KILLED',
    'MOTORISTS_INJURED', 'MOTORISTS_KILLED',
    'ON_STREET_NAME', 'CROSS_STREET_NAME', 'OFF_STREET_NAME',
    'REPORTED_ZIPCODE', 'REPORTED_BOROUGH'
]

# Ensure all columns exist and handle geometry column
final_df = final_df[collisions_columns].copy()

# Convert to regular DataFrame (drop geometry)
final_df = pd.DataFrame(final_df)

print("Final collisions table schema:")
print(final_df.dtypes)

In [ ]:
# Filter causes_types to only include records in the final dataset
vehicle_causes = ct_df[ct_df['UNIQUE_KEY'].isin(final_df['UNIQUE_KEY'])].copy()
vehicle_causes = vehicle_causes.reset_index(drop=True)

print(f"Final causes_types table: {len(vehicle_causes):,} rows")
print(f"Average vehicles per collision: {len(vehicle_causes) / len(final_df):.2f}")

### Contributing Factor & Vehicle Type Analysis

Examine the distribution of contributing factors and vehicle types before export.
This helps validate the normalization and identify any remaining data quality issues.

In [ ]:
# Contributing factor distribution
print("=== Top 20 Contributing Factors ===")
cause_counts = vehicle_causes['CAUSE'].value_counts()
print(cause_counts.head(20))

print(f"\nTotal unique contributing factors: {vehicle_causes['CAUSE'].nunique()}")
print(f"\nRecords with 'UNSPECIFIED': {(vehicle_causes['CAUSE'] == 'UNSPECIFIED').sum():,}")

In [ ]:
# Vehicle type distribution after normalization
print("=== Top 20 Vehicle Types (After Normalization) ===")
type_counts = vehicle_causes['VEHICLE_TYPE'].value_counts()
print(type_counts.head(20))

print(f"\nTotal unique vehicle types: {vehicle_causes['VEHICLE_TYPE'].nunique()}")
print(f"\nRecords with 'UNKNOWN': {(vehicle_causes['VEHICLE_TYPE'] == 'UNKNOWN').sum():,}")

In [ ]:
# Visualize top contributing factors
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 10 causes
cause_counts.head(10).plot(
    kind='barh',
    ax=axes[0],
    color='steelblue'
)
axes[0].set_title('Top 10 Contributing Factors')
axes[0].set_xlabel('Number of Records')
axes[0].invert_yaxis()

# Top 10 vehicle types
type_counts.head(10).plot(
    kind='barh',
    ax=axes[1],
    color='darkgreen'
)
axes[1].set_title('Top 10 Vehicle Types')
axes[1].set_xlabel('Number of Records')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# Cross-tabulation: Top causes by vehicle type
print("=== Contributing Factors by Vehicle Type (Top 5 each) ===")
top_causes = cause_counts.head(5).index.tolist()
top_types = type_counts.head(5).index.tolist()

crosstab = pd.crosstab(
    vehicle_causes[vehicle_causes['CAUSE'].isin(top_causes)]['CAUSE'],
    vehicle_causes[vehicle_causes['VEHICLE_TYPE'].isin(top_types)]['VEHICLE_TYPE']
)
print(crosstab)

In [ ]:
# Vehicles per collision distribution
vehicles_per_collision = vehicle_causes.groupby('UNIQUE_KEY').size()

print("=== Vehicles per Collision ===")
print(vehicles_per_collision.value_counts().sort_index())

print(f"\nAverage vehicles per collision: {vehicles_per_collision.mean():.2f}")
print(f"Max vehicles in a single collision: {vehicles_per_collision.max()}")

## 7. Export to BigQuery

Upload the processed data to Google BigQuery with proper schema definitions including column descriptions.

In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Create dataset if it doesn't exist
dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"
try:
    client.get_dataset(DATASET_ID)
    print(f"Dataset '{DATASET_ID}' already exists")
except Exception:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    client.create_dataset(dataset, exists_ok=True)
    print(f"Created dataset '{DATASET_ID}'")

In [ ]:
# Define schema for collisions table with column descriptions
collisions_schema = [
    SchemaField("UNIQUE_KEY", "STRING", mode="REQUIRED",
                description="Unique identifier for the collision"),
    SchemaField("DATE_TIME", "TIMESTAMP", mode="REQUIRED",
                description="Date and time of the collision"),
    SchemaField("LATITUDE", "FLOAT",
                description="Latitude of the collision location"),
    SchemaField("LONGITUDE", "FLOAT",
                description="Longitude of the collision location"),
    SchemaField("ZIPCODE", "STRING",
                description="ZIP code (detected from coordinates, or reported)"),
    SchemaField("NEIGHBORHOOD", "STRING",
                description="NYC neighborhood (detected from coordinates)"),
    SchemaField("BOROUGH", "STRING",
                description="NYC borough (detected from coordinates, or reported)"),
    SchemaField("PERSONS_INJURED", "INTEGER",
                description="Total number of persons injured"),
    SchemaField("PERSONS_KILLED", "INTEGER",
                description="Total number of persons killed"),
    SchemaField("PEDESTRIANS_INJURED", "INTEGER",
                description="Number of pedestrians injured"),
    SchemaField("PEDESTRIANS_KILLED", "INTEGER",
                description="Number of pedestrians killed"),
    SchemaField("CYCLISTS_INJURED", "INTEGER",
                description="Number of cyclists injured"),
    SchemaField("CYCLISTS_KILLED", "INTEGER",
                description="Number of cyclists killed"),
    SchemaField("MOTORISTS_INJURED", "INTEGER",
                description="Number of motorists injured"),
    SchemaField("MOTORISTS_KILLED", "INTEGER",
                description="Number of motorists killed"),
    SchemaField("ON_STREET_NAME", "STRING",
                description="Name of the street where the collision occurred"),
    SchemaField("CROSS_STREET_NAME", "STRING",
                description="Name of the cross street"),
    SchemaField("OFF_STREET_NAME", "STRING",
                description="Name of the off-street location"),
    SchemaField("REPORTED_ZIPCODE", "STRING",
                description="Originally reported ZIP code"),
    SchemaField("REPORTED_BOROUGH", "STRING",
                description="Originally reported borough"),
]

In [ ]:
# Define schema for causes_types table
causes_types_schema = [
    SchemaField("UNIQUE_KEY", "STRING", mode="REQUIRED",
                description="Collision ID (foreign key to collisions table)"),
    SchemaField("CAUSE", "STRING",
                description="Contributing factor for the vehicle"),
    SchemaField("VEHICLE_TYPE", "STRING",
                description="Type of vehicle involved"),
    SchemaField("VEHICLE", "INTEGER",
                description="Vehicle number in the collision (1-5)"),
]

In [ ]:
%%time
# Upload collisions table
collisions_table_id = f"{PROJECT_ID}.{DATASET_ID}.collisions"
job_config = bigquery.LoadJobConfig(
    schema=collisions_schema,
    write_disposition='WRITE_TRUNCATE'  # Replace existing table
)

job = client.load_table_from_dataframe(final_df, collisions_table_id, job_config=job_config)
job.result()  # Wait for completion

print(f"Loaded {job.output_rows:,} rows into {collisions_table_id}")

In [ ]:
%%time
# Upload causes_types table
causes_types_table_id = f"{PROJECT_ID}.{DATASET_ID}.causes_types"
job_config = bigquery.LoadJobConfig(
    schema=causes_types_schema,
    write_disposition='WRITE_TRUNCATE'  # Replace existing table
)

job = client.load_table_from_dataframe(vehicle_causes, causes_types_table_id, job_config=job_config)
job.result()  # Wait for completion

print(f"Loaded {job.output_rows:,} rows into {causes_types_table_id}")

In [ ]:
# Add primary key constraint (not enforced, for documentation)
# Run these SQL commands in BigQuery console or uncomment to execute:
pk_fk_sql = '''
-- Define primary key for collisions table (not enforced)
ALTER TABLE `{project}.{dataset}.collisions`
ADD PRIMARY KEY (UNIQUE_KEY) NOT ENFORCED;

-- Define foreign key for causes_types table (not enforced)
ALTER TABLE `{project}.{dataset}.causes_types`
ADD FOREIGN KEY (UNIQUE_KEY)
REFERENCES `{project}.{dataset}.collisions`(UNIQUE_KEY) NOT ENFORCED;
'''.format(project=PROJECT_ID, dataset=DATASET_ID)

print("SQL for primary/foreign key constraints:")
print(pk_fk_sql)

In [ ]:
# Grant read access to all authenticated users
dataset = client.get_dataset(DATASET_ID)
access_entries = list(dataset.access_entries)

# Check if already granted
already_granted = any(
    e.entity_id == 'allAuthenticatedUsers'
    for e in access_entries
)

if not already_granted:
    access_entries.append(
        bigquery.AccessEntry(
            role='READER',
            entity_type='specialGroup',
            entity_id='allAuthenticatedUsers',
        )
    )
    dataset.access_entries = access_entries
    client.update_dataset(dataset, ['access_entries'])
    print(f"Granted read access to allAuthenticatedUsers")
else:
    print("Read access already granted to allAuthenticatedUsers")

## 8. Pipeline Complete

In [ ]:
# Calculate total runtime
t_end = datetime.now()
runtime = t_end - t_start

# Calculate retention statistics
original_count = ORIGINAL_RECORD_COUNT  # Original loaded data
final_count = len(final_df)  # After all filtering
retention_pct = 100 * final_count / original_count

print("=" * 70)
print("NYC COLLISIONS DATA PIPELINE COMPLETE")
print("=" * 70)

print(f"\n⏱️  RUNTIME")
print(f"   Started:  {t_start}")
print(f"   Finished: {t_end}")
print(f"   Duration: {runtime}")

print(f"\n📊 DATA RETENTION")
print(f"   Original records loaded:     {original_count:>12,}")
print(f"   Records excluded (quality):  {len(records_to_exclude):>12,}")
print(f"   Records excluded (geocoding):{original_count - len(gdf_nyc):>12,}")
print(f"   Records excluded (duplicates):{len(duplicate_keys):>12,}")
print(f"   ─────────────────────────────────────────────")
print(f"   Final records exported:      {final_count:>12,}  ({retention_pct:.1f}%)")

print(f"\n📁 BIGQUERY TABLES")
print(f"   {PROJECT_ID}.{DATASET_ID}.collisions     → {len(final_df):,} rows")
print(f"   {PROJECT_ID}.{DATASET_ID}.causes_types   → {len(vehicle_causes):,} rows")

print(f"\n🔍 SAMPLE QUERY")
print(f"""
   SELECT
     DATE(DATE_TIME) as date,
     BOROUGH,
     SUM(PERSONS_INJURED) as total_injured,
     SUM(PERSONS_KILLED) as total_killed
   FROM `{PROJECT_ID}.{DATASET_ID}.collisions`
   GROUP BY 1, 2
   ORDER BY total_killed DESC
   LIMIT 10
""")
print("=" * 70)